In [17]:
# DATA INPUT SCRIPT



# THIS SCRIPT IS RUNNING SCRIPTS FROM. https://rapidapi.com/coingecko/api/coingecko?endpoint=apiendpoint_4125add1-6b8e-4d2d-ade7-f6d2275b6a9b

#HOW TO USE THIS
#1. change G_save_path to your direcory where you would like the files to go
#2. Run the code!

# HOW TO CONTRIBUTE
# A. New data scripts: create new scripts and put them in the root (set all output files to go to /data)
# B. New analysis: create new scripts and put them in the /analysis

#3. Have fun - Please contribute!
# ***********************************************************************************
import os.path
from datetime import datetime
# GLOBAL VARS
save_path = '/Users/bob/Documents/dev/projects/python/cryptoData/data'
env_path = '/Users/bob/Documents/dev/projects/python/cryptoData/env'
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
# UTILTIY FUNCTIONS
def removeCurrent(fileName):
    # used in outputToFile()
    completeName = os.path.join(save_path, fileName)
    if os.path.exists(completeName):
        os.remove(completeName)
        return 'File: Overwritten'
    else:
        return "File: Created"
    
def JSONExport(mylist,fileName): 
    completeName = os.path.join(save_path, fileName)  
    overwriteStatus = removeCurrent(completeName)
    with open(completeName, 'w') as f:
        json.dump(mylist, f)
    print('Records Exported:',len(mylist),'records' )
    print(current_time, 'Exported file:',completeName,overwriteStatus,)

def JSONImport(fileName): 
    completeName = os.path.join(save_path, fileName) 
    print(current_time,'Imported file:',completeName) 
    with open(completeName) as f:
        result = json.load(f)
    return result 
def JSONImportEnv(envVar): 
    fileName = 'env.json'
    completeName = os.path.join(env_path, fileName) 
    print(current_time,'Imported Env:',completeName) 
    with open(completeName) as f:
        result = json.load(f)
    return result[0][envVar] 



#PROGRAM
print( current_time,'Utilty functions generated..')

12:35:27 Utilty functions generated..


In [18]:
# A GET COINS
# 1. Get All the coins
# 2. Create a paged list of the coins (so that I can hit API with many small requests)
import requests
import json



#import pandas as pd 

# 1. Get All the coins
def getCoins():
    def createCoinList(jsonData):
        result = []
        for n in range(len(jsonData)):
            result.append(jsonData[n]['id'])
        return result
    def fetchData():
        url = "https://coingecko.p.rapidapi.com/coins/list"
        headers = {
            'x-rapidapi-key': rapidapiKey,
            'x-rapidapi-host': "coingecko.p.rapidapi.com"
            }
        response = requests.request("GET", url, headers=headers)
        jsonData = json.loads(response.text)
        return jsonData
    #RUN
    jsonData = fetchData()
    #print(jsonData)
    result = createCoinList(jsonData)
    # print('Total Number of coins fetched:',len(result))
    # print('ALL COINS',result)
    return result

# 2. Create a paged list of the coins
def divide_chunks(myList, page_number):
    def yield_chunks(myList, page_number):
        # looping till length l
        for i in range(0, len(myList), page_number): 
            yield myList[i:i + page_number]
    result = list(yield_chunks(all_coins, page_number))
    print (current_time,'pagedCoins created: Pages Created:', len(result))
    return result

#PROGRAM
rapidapiKey = JSONImportEnv('x-rapidapi-key')
all_coins = getCoins()
outputPagedCoins = divide_chunks(all_coins,200)
JSONExport(outputPagedCoins,'pagedCoins.json')


12:35:27 Imported Env: /Users/bob/Documents/dev/projects/python/cryptoData/env/env.json
12:35:27 pagedCoins created: Pages Created: 34
Records Exported: 34 records
12:35:27 Exported file: /Users/bob/Documents/dev/projects/python/cryptoData/data/pagedCoins.json File: Overwritten


In [19]:
# B. GET STATS 
import time
import requests
import random


def createQueryString(myList):
    # convert list format: ['bitcoin','etherium'] to string: 'bitcoin,etherium'
    #used in fetchStatsAll()
    def removeUnwantedChars(myString):
        no_leftBrackets =  myString.replace("[", "")
        no_rightBrackets = no_leftBrackets.replace("]", "")
        no_quotes = no_rightBrackets.replace("'", "")
        result = no_quotes
        return result
    myString = str(myList)
    result = removeUnwantedChars(myString)
    return result
def fetchStats(coins):
    #used in fetchStatsAll()
    url = "https://coingecko.p.rapidapi.com/coins/markets"
    querystring = {"vs_currency":"usd","ids":coins,"per_page":"250","order":"market_cap_desc"}
    headers = {
        'x-rapidapi-key': rapidapiKey,
        'x-rapidapi-host': "coingecko.p.rapidapi.com"
        }
    response = requests.request("GET", url, headers=headers, params=querystring)
    jsonData = json.loads(response.text)
    return jsonData
#run fetchStats n times
def fetchStatsAll(pagedCoins):
    #fetch the stats for the coins 1 page at a time (to overcome API limit)
    fetchedCollated=[]
    for pageNumber in range(len(pagedCoins)):
        #get coins for this api fetch
        queryString = createQueryString(pagedCoins[pageNumber])
        pageStats = fetchStats(queryString)
        fetchedCollated.append(pageStats)
        random_number = random.randint(1,5)
        time.sleep(random_number)
        print(current_time,len(pagedCoins[pageNumber]),'Stats Fetched...' )
    result = fetchedCollated
    return  result

# this doesn't work becuase it just puts the string back in
def removePages(pagedList):
    result = []
    for n in range(len(pagedList)):
            result.extend(pagedList[n])
    return result

# PROGRAM 
pagedCoins = JSONImport('pagedCoins.json')
pagedStatsOutput = fetchStatsAll(pagedCoins)
statsOutput = removePages(pagedStatsOutput)
JSONExport(statsOutput,'pageStats.json')




12:35:27 Imported file: /Users/bob/Documents/dev/projects/python/cryptoData/data/pagedCoins.json
12:35:27 200 Stats Fetched...
12:35:27 200 Stats Fetched...
12:35:27 200 Stats Fetched...
12:35:27 200 Stats Fetched...
12:35:27 200 Stats Fetched...
12:35:27 200 Stats Fetched...
12:35:27 200 Stats Fetched...
12:35:27 200 Stats Fetched...
12:35:27 200 Stats Fetched...
12:35:27 200 Stats Fetched...
12:35:27 200 Stats Fetched...
12:35:27 200 Stats Fetched...
12:35:27 200 Stats Fetched...
12:35:27 200 Stats Fetched...
12:35:27 200 Stats Fetched...
12:35:27 200 Stats Fetched...
12:35:27 200 Stats Fetched...
12:35:27 200 Stats Fetched...


JSONDecodeError: Expecting value: line 1 column 1 (char 0)